In [23]:
import numpy as np
import random

# Q-learning 参数
alpha = 0.9                # 学习率
gamma = 0.9                # 折扣因子
delta = 5e-4               # 状态变化量
G_ref = 15                 # 目标函数 G 的参考值
G_min = 20                 # 最小目标函数值
N_T = int(1e5)             # 最大训练回合数
N_i = 5000                 # 每个优化过程的训练次数
M = int(1e4)               # 基于 ε-greedy 法的探索次数



# DAB DC-DC 转换器电路参数
n = 1                      # 变压器匝数比
V1 = 140                   # 输入电压
V2 = 50                    # 输出电压
P_o_star = 200               # 传输功率
fs = 50e3                  # 开关频率，单位：Hz
Lk = 41e-6                 # 串联电感，单位：H
P_max = n * V1 * V2 / (8 * fs * Lk)  # 最大传输功率
P_on = P_o_star / P_max    # 归一化传输功率
k = V1 / (n * V2)


# 计算归一化系数
normalization_factor = n * V2 / (4 * fs * Lk)

# 离散化步长和状态空间大小
num_bins = int(2 / delta) + 1  # 4001 个离散化值

# 动作空间：增量变化
actions = [(d1, d2, d3) for d1 in [0, delta, -delta] for d2 in [0, delta, -delta] for d3 in [0, delta, -delta]]
# 离散化函数：将连续的 D1, D2, D3 映射为离散的整数索引
def discretize(value):
    # 将 value 从 [-1, 1] 范围映射到 [0, num_bins - 1] 的索引
    index = int(round((value + 1) / delta))
    return max(0, min(index, num_bins - 1))

# 根据离散化后的 D1, D2, D3 生成单一索引
def get_state_index(D1, D2, D3):
    D1_index = discretize(D1)
    D2_index = discretize(D2)
    D3_index = discretize(D3)
    return D1_index * num_bins * num_bins + D2_index * num_bins + D3_index

# 使用字典初始化 Q 表
Q_table = {}

# Q 值更新函数
def update_q_value(state_index, action, reward, next_state_index):
    # 如果 Q 值不存在，初始化为 0
    if state_index not in Q_table:
        Q_table[state_index] = np.zeros(len(actions))
    if next_state_index not in Q_table:
        Q_table[next_state_index] = np.zeros(len(actions))
    
    best_next_action = np.argmax(Q_table[next_state_index])
    td_target = reward + gamma * Q_table[next_state_index][best_next_action]
    td_error = td_target - Q_table[state_index][action]
    Q_table[state_index][action] += alpha * td_error

def determine_mode(D1, D2, D3):
    # 根据提供的不等式条件选择模式
    if D1 >= D2 and 0 <= D3 <= (D1 - D2):
        return "mode_1"
    elif D1 >= D2 and 0 <= D3 <= (D1 - D2):
        return "mode_1_prime"
    elif D2 >= D1 and (1 + D1 - D2) <= D3 <= 1:
        return "mode_2"
    elif D2 >= D1 and (1 + D1 - D2) <= D3 <= 1:
        return "mode_2_prime"
    elif D2 <= (1 - D1) and D1 <= D3 <= (1 - D2):
        return "mode_3"
    elif D2 <= (1 - D1) and D1 <= D3 <= (1 - D2):
        return "mode_3_prime"
    elif D1 <= D3 <= 1 and (1 - D3) <= D2 <= (1 - D3 + D1):
        return "mode_4"
    elif D1 <= D3 <= 1 and (1 - D3) <= D2 <= (1 - D3 + D1):
        return "mode_4_prime"
    elif (D1 - D3) <= D2 <= (1 - D3) and 0 <= D3 <= D1:
        return "mode_5"
    elif (D1 - D3) <= D2 <= (1 - D3) and 0 <= D3 <= D1:
        return "mode_5_prime"
    elif (1 - D2) <= D1 and (1 - D2) <= D3 <= D1:
        return "mode_6"
    elif (1 - D2) <= D1 and (1 - D2) <= D3 <= D1:
        return "mode_6_prime"
    else:
        # 默认模式，如果没有任何匹配条件
        return "mode_1"

# 计算传输功率，返回归一化后的结果
def calculate_power(mode, D1, D2, D3):
    if mode == "mode_1":
        P_o = 2 * (D2**2 - D1 * D2 + 2 * D2 * D3)
    elif mode == "mode_1_prime":
        P_o = -2 * (D2**2 - D1 * D2 + 2 * D2 * D3)
    elif mode == "mode_2":
        P_o = 2 * (D1**2 - D1 * D2 + 2 * D1 - 2 * D1 * D3)
    elif mode == "mode_2_prime":
        P_o = -2 * (D1**2 - D1 * D2 + 2 * D1 - 2 * D1 * D3)
    elif mode == "mode_3":
        P_o = 2 * D2 * D3
    elif mode == "mode_3_prime":
        P_o = -2 * D2 * D3
    elif mode == "mode_4":
        P_o = 2 * (-D2**2 - D3**2 + 2 * D2 * D3 + 2 * D3 + 2 * D2 * D3 + D1 * D2 - 1)
    elif mode == "mode_4_prime":
        P_o = -2 * (-D2**2 - D3**2 + 2 * D2 * D3 + 2 * D3 + 2 * D2 * D3 + D1 * D2 - 1)
    elif mode == "mode_5":
        P_o = 2 * (D1 - D3) * D2 + D3 + 2 * D2 * D3
    elif mode == "mode_5_prime":
        P_o = -2 * (D1 - D3) * D2 + D3 + 2 * D2 * D3
    elif mode == "mode_6":
        P_o = 2 * (-D1**2 - D2**2 - D3**2 + 2 * D3 * D2 + D1 + D2 + 2 * D2 * D3 + 2 * D1 - 1)
    elif mode == "mode_6_prime":
        P_o = -2 * (-D1**2 - D2**2 - D3**2 + 2 * D3 * D2 + D1 + D2 + 2 * D2 * D3 + 2 * D1 - 1)
    else:
        P_o = 0
    return P_o

# 计算峰值电流并归一化
def calculate_peak_current(mode, D1, D2, D3):
    if mode == "mode_1":
        currents = [
            -k * D1 + D2,
            -k * D1 + 2 * k * D3 + D2,
            -k * D1 + 2 * k * D2 + 2 * k * D3 - D2,
            k * D1 - D2
        ]
    elif mode == "mode_1_prime":
        currents = [
            -k * D1 - D2,
            -k * D1 + 2 * k * D3 - D2,
            -k * D1 + 2 * k * D2 + 2 * k * D3 + D2,
            k * D1 + D2
        ]
    elif mode == "mode_2":
        currents = [
            -k * D1 + 2 * D1 - D2 - 2 * D3,
            k * D1 + 2 * D1 - D2 + 2 * D3,
            k * D1 + D2,
            k * D1 + D2
        ]
    elif mode == "mode_2_prime":
        currents = [
            -k * D1 - 2 + D2 + 2 * D3,
            k * D1 - 2 + D2 + 2 * D3,
            k * D1 - D2,
            k * D1 - D2
        ]
    elif mode == "mode_3":
        currents = [
            -k * D1 + D2,
            k * D1 + D2,
            k * D1 + D2,
            k * D1 + D2
        ]
    elif mode == "mode_3_prime":
        currents = [
            -k * D1 - D2,
            k * D1 - D2,
            k * D1 - D2,
            k * D1 + D2
        ]
    elif mode == "mode_4":
        currents = [
            -k * D1 + 2 + D2 - 2 * D3,
            -k * D1 - 2 * k * D2 + 2 * k * D3 + D2,
            k * D1 + D2,
            k * D1 - D2
        ]
    elif mode == "mode_4_prime":
        currents = [
            -k * D1 - 2 + D2 + 2 * D3,
            k * D1 - 2 + k * D2 - k * D3,
            k * D1 - D2,
            k * D1 - D2
        ]
    elif mode == "mode_5":
        currents = [
            -k * D1 + D2,
            k * D1 - 2 * D1 + D2 + 2 * D3,
            k * D1 - D2,
            k * D1 - D2
        ]
    elif mode == "mode_5_prime":
        currents = [
            -k * D1 - D2,
            k * D1 + 2 * D1 - D2 - 2 * D3,
            k * D1 - D2,
            k * D1 + D2
        ]
    elif mode == "mode_6":
        currents = [
            -k * D1 - D2 - 2 * D3 + 2,
            -k * D1 + 2 * k * D3 + D2 - k,
            k * D1 - 2 * D1 + D2 + 2 * D3,
            k * D1 - 2 * D1 + D2 + 2 * D3
        ]
    elif mode == "mode_6_prime":
        currents = [
            -k * D1 + D2 + 2 * D3 - 2,
            -k * D1 + 2 * k * D2 + 2 * k * D3 - D2,
            k * D1 + 2 * D1 - D2 + 2 * D3,
            k * D1 + D2
        ]
    else:
        currents = [0]

    # 取绝对值最大的电流作为峰值电流，并进行归一化
    peak_current = max(abs(i) for i in currents) # * normalization_factor
    return peak_current, currents  # 返回电流列表以用于 ZVS 检查

# 检查当前状态是否满足给定模式的 ZVS 约束
def check_mode_constraints(mode, currents):
    # 根据模式定义 ZVS 约束条件
    if mode == "mode_1":
        return currents[0] <= 0 and currents[1] == 0 and currents[2] == 0
    elif mode == "mode_1_prime":
        return currents[0] <= 0 and currents[1] <= 0 and currents[2] >= 0
    elif mode == "mode_2":
        return currents[0] <= 0 and currents[1] >= 0 and currents[2] >= 0
    elif mode == "mode_2_prime":
        return currents[0] <= 0 and currents[1] >= 0 and currents[2] <= 0
    elif mode == "mode_3":
        return currents[0] == 0 and currents[1] >= 0
    elif mode == "mode_3_prime":
        return currents[0] <= 0 and currents[1] == 0
    elif mode == "mode_4":
        return currents[0] <= 0 and currents[1] >= 0 and currents[2] >= 0
    elif mode == "mode_4_prime":
        return currents[0] <= 0 and currents[1] <= 0 and currents[2] == 0
    elif mode == "mode_5":
        return currents[0] == 0 and currents[1] >= 0 and currents[2] >= 0
    elif mode == "mode_5_prime":
        return currents[0] <= 0 and currents[1] <= 0 and currents[2] >= 0
    elif mode == "mode_6":
        return currents[0] <= 0 and currents[1] >= 0 and currents[2] >= 0 and currents[3] >= 0
    elif mode == "mode_6_prime":
        return currents[0] <= 0 and currents[1] <= 0 and currents[2] <= 0 and currents[3] >= 0
    else:
        return False


# 奖励函数
def get_reward(G_c, G_p):
    delta_G = G_c - G_p
    if G_c <= G_min:
        return 20
    elif delta_G >= 0 and delta_G < G_ref:
        return -abs(delta_G) / G_ref
    elif delta_G < 0:
        return 1
    else:
        return -1

# 计算目标函数 G
def calculate_objective_function(I_p, P_o, zvs_satisfied, beta_zvs=1, beta_non_zvs=10, lambda_=50):
    beta = beta_zvs if zvs_satisfied else beta_non_zvs
    power_error = abs(P_o - P_on)
    return beta * I_p + lambda_ * power_error

# Q-learning 的 ε-greedy 策略
def epsilon_greedy_policy(state_index, epsilon):
    if random.uniform(0, 1) < epsilon:
        return random.randint(0, len(actions) - 1)
    else:
        return np.argmax(Q_table.get(state_index, np.zeros(len(actions))))

# 将状态索引转换回 (D1, D2, D3)
def get_d_values_from_index(state_index, delta, D_min=-1):
    num_bins = int(2 / delta) + 1
    D1_index = state_index // (num_bins * num_bins)
    D2_index = (state_index % (num_bins * num_bins)) // num_bins
    D3_index = state_index % num_bins

    D1 = D_min + D1_index * delta
    D2 = D_min + D2_index * delta
    D3 = D_min + D3_index * delta
    return D1, D2, D3

In [ ]:
def train():
    global G_min  # 声明 G_min 为全局变量
    min_objective_value = float('inf')  # 初始最小目标函数值设为正无穷
    min_objective_state = None  # 用于存储目标函数值最大的状态

    for episode in range(N_T):
         # 每个 episode 随机初始化移相角 (D1, D2, D3) 在 [-1, 1] 范围内
        D1 = round(random.uniform(-1, 1) / delta) * delta
        D2 = round(random.uniform(-1, 1) / delta) * delta
        D3 = round(random.uniform(-1, 1) / delta) * delta

        # 根据初始 (D1, D2, D3) 计算初始模式
        mode = determine_mode(D1, D2, D3)
        
        total_reward = 0
        consecutive_G_min_count = 0  # 用于统计连续满足 G <= G_min 的次数

        # 初始化状态的目标函数值
        I_p, currents = calculate_peak_current(mode, D1, D2, D3)
        P_o = calculate_power(mode, D1, D2, D3)
        zvs_satisfied = check_mode_constraints(mode, currents)
        G_p = calculate_objective_function(I_p, P_o, zvs_satisfied)

        for step in range(N_i):
            epsilon = max(0.1, 1.0 - episode / M)  # 逐步减小 ε，增加利用率
            state_index = get_state_index(D1, D2, D3)  # 使用离散化后的单一索引
            if state_index not in Q_table:
                Q_table[state_index] = np.zeros(len(actions))
                
            # 选择动作
            action_index = epsilon_greedy_policy(state_index, epsilon)
            action = actions[action_index]
            
            # 更新状态
            new_D1, new_D2, new_D3 = D1 + action[0], D2 + action[1], D3 + action[2]
            
            # 根据更新后的 (D1, D2, D3) 重新计算 mode
            mode = determine_mode(new_D1, new_D2, new_D3)
            
            # 计算峰值电流、电流应力和目标函数值
            I_p, new_currents = calculate_peak_current(mode, new_D1, new_D2, new_D3)
            P_o = calculate_power(mode, new_D1, new_D2, new_D3)
            zvs_satisfied = check_mode_constraints(mode, new_currents)
            G_c = calculate_objective_function(I_p, P_o, zvs_satisfied)  # 当前状态的目标函数

            # 检查是否是当前最小目标函数值
            if G_c < min_objective_value:
                min_objective_value = G_c
                min_objective_state = (new_D1, new_D2, new_D3)

            # 计算奖励
            reward = get_reward(G_c, G_p)
            total_reward += reward

            # 使用离散化后的单一索引更新 Q 值
            next_state_index = get_state_index(new_D1, new_D2, new_D3)
            update_q_value(state_index, action_index, reward, next_state_index)

            # 更新前一状态的目标函数值为当前的 G_c
            G_p = G_c

            # 判断是否达到目标
            if G_c <= G_min:
                G_min = G_c
                consecutive_G_min_count += 1
                if consecutive_G_min_count >= M:
                    print(f"算法在第 {episode} 回合收敛，提前结束训练")
                    return Q_table, min_objective_state, min_objective_value
            else:
                consecutive_G_min_count = 0
            
            # 更新状态
            D1, D2, D3 = new_D1, new_D2, new_D3

        # 每隔 100 个回合打印进度
        if episode % 100 == 0:
            print(f"Episode {episode}, Total Reward: {total_reward}, G_c: {G_c}")

        # 如果达到最大训练次数，结束训练
        if episode >= N_T - 1:
            print("达到最大训练回合数，结束训练")
            return Q_table, min_objective_state, min_objective_value

# 训练并输出 Q 表
Q_table, min_objective_state, min_objective_value = train()




Episode 0, Total Reward: 2392.547796666676, G_c: 44.365521428571526
Episode 100, Total Reward: 2384.794206666673, G_c: 70.46872142857147
Episode 200, Total Reward: 2385.108206190474, G_c: 24.24422142857134
Episode 300, Total Reward: 2399.3577100000057, G_c: 121.55312857143036
Episode 400, Total Reward: 2316.491965000003, G_c: 49.670171428571095
Episode 500, Total Reward: 2221.9923516666704, G_c: 269.49180357142654
Episode 600, Total Reward: 2275.848385000004, G_c: 23.20527142857143
Episode 700, Total Reward: 2281.350530000008, G_c: 36.62632142857142
Episode 800, Total Reward: 2216.655601666662, G_c: 42.41757142857143
Episode 900, Total Reward: 2218.1821750000026, G_c: 99.82817857142916
Episode 1000, Total Reward: 2200.124281666667, G_c: 69.20262857142842
Episode 1100, Total Reward: 2147.743453333336, G_c: 51.610178571429174
Episode 1200, Total Reward: 2185.3655916666676, G_c: 155.27227857143146
Episode 1300, Total Reward: 2144.8053599999953, G_c: 64.75232857142866
Episode 1400, Total R

In [20]:
# 比较 Q-learning 和枚举法的最优解
if min_objective_state is not None:
    D1_q, D2_q, D3_q = min_objective_state
    print(f"\nQ-learning 对应的最优组合 (D1, D2, D3): ({D1_q:.3f}, {D2_q:.3f}, {D3_q:.3f})")
    print(f"Q-learning 找到的最小目标函数值: {min_objective_value:.3f}")

def test(D1,D2,D3):
    # 根据当前组合的 (D1, D2, D3) 确定 mode
    mode = determine_mode(D1, D2, D3)
    
    # 计算目标函数值
    I_p, currents = calculate_peak_current(mode, D1, D2, D3)
    P_o = calculate_power(mode, D1, D2, D3)
    zvs_satisfied = check_mode_constraints(mode, currents)
    G = calculate_objective_function(I_p, P_o, zvs_satisfied)
    return G

print(f"\n{test(0.15,0.42,0.56)}")

def find_min_objective_by_enumeration(delta=1e-2, D_min=-1, D_max=1):
    min_objective_value = float('inf')
    best_combination_enum = (0, 0, 0)
    
    # 使用四舍五入以减少浮点数误差
    D1_values = np.round(np.arange(D_min, D_max + delta, delta), decimals=3)
    D2_values = np.round(np.arange(D_min, D_max + delta, delta), decimals=3)
    D3_values = np.round(np.arange(D_min, D_max + delta, delta), decimals=3)

    total_combinations = len(D1_values) * len(D2_values) * len(D3_values)
    print(f"总组合数: {total_combinations}")

    combination_count = 0  # 组合计数器，用于跟踪进度

    for D1 in D1_values:
        for D2 in D2_values:
            for D3 in D3_values:
                combination_count += 1
                
                # 根据当前组合的 (D1, D2, D3) 确定 mode
                mode = determine_mode(D1, D2, D3)
                
                # 计算目标函数值
                I_p, currents = calculate_peak_current(mode, D1, D2, D3)
                P_o = calculate_power(mode, D1, D2, D3)
                zvs_satisfied = check_mode_constraints(mode, currents)
                G = calculate_objective_function(I_p, P_o, zvs_satisfied)

                # 使用接近判断方法检查是否到达预期值
                if np.isclose(D1, 0.150, atol=1e-3) and np.isclose(D2, 0.420, atol=1e-3) and np.isclose(D3, 0.560, atol=1e-3):
                    print(f"\n找到目标组合 (D1, D2, D3): ({D1:.3f}, {D2:.3f}, {D3:.3f})")
                    print(f"G 值: {G}, 当前最小值: {min_objective_value}")

                # 如果找到更小的目标函数值，则更新最优组合
                if G < min_objective_value: 
                    min_objective_value = G
                    best_combination_enum = (D1, D2, D3)

                # 每隔 50,000 次输出进度
                if combination_count % 50000 == 0:
                    print(f"已完成 {combination_count}/{total_combinations} 组合 ({(combination_count / total_combinations) * 100:.2f}%)")

    # 输出枚举法找到的最终结果
    best_D1, best_D2, best_D3 = best_combination_enum
    print(f"\n枚举法找到的最优组合 (D1, D2, D3): ({best_D1:.3f}, {best_D2:.3f}, {best_D3:.3f})")
    print(f"找到的最小目标函数值: {min_objective_value:.3f}")
    return best_combination_enum, min_objective_value

# 使用枚举法找到的目标函数最小的组合
best_combination_enum, min_objective_enum = find_min_objective_by_enumeration()




Q-learning 对应的最优组合 (D1, D2, D3): (0.150, 0.420, 0.560)
Q-learning 找到的最小目标函数值: 0.931

0.9314285714285756
总组合数: 8120601
已完成 50000/8120601 组合 (0.62%)
已完成 100000/8120601 组合 (1.23%)
已完成 150000/8120601 组合 (1.85%)
已完成 200000/8120601 组合 (2.46%)
已完成 250000/8120601 组合 (3.08%)
已完成 300000/8120601 组合 (3.69%)
已完成 350000/8120601 组合 (4.31%)
已完成 400000/8120601 组合 (4.93%)
已完成 450000/8120601 组合 (5.54%)
已完成 500000/8120601 组合 (6.16%)
已完成 550000/8120601 组合 (6.77%)
已完成 600000/8120601 组合 (7.39%)
已完成 650000/8120601 组合 (8.00%)
已完成 700000/8120601 组合 (8.62%)
已完成 750000/8120601 组合 (9.24%)
已完成 800000/8120601 组合 (9.85%)
已完成 850000/8120601 组合 (10.47%)
已完成 900000/8120601 组合 (11.08%)
已完成 950000/8120601 组合 (11.70%)
已完成 1000000/8120601 组合 (12.31%)
已完成 1050000/8120601 组合 (12.93%)
已完成 1100000/8120601 组合 (13.55%)
已完成 1150000/8120601 组合 (14.16%)
已完成 1200000/8120601 组合 (14.78%)
已完成 1250000/8120601 组合 (15.39%)
已完成 1300000/8120601 组合 (16.01%)
已完成 1350000/8120601 组合 (16.62%)
已完成 1400000/8120601 组合 (17.24%)
已完成 1450000/8120601 组